# Audio Classification with SpeechBrain and Cleanlab

In this quickstart tutorial, we will use Cleanlab to find label issues in the Spoken Digit dataset (it's like MNIST for audio). The dataset contains 2,500 audio clips with English pronunciations of the digits 0 to 9 (these are the labels to predict from the audio).

**Overview of what we'll do in this tutorial:**

- Extract features from audio clips (.wav files) using a pre-trained Pytorch model (available via HuggingFace) that was previously fit to the [VoxCeleb](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/) speech dataset.

- Train a cross-validated linear model using the extracted features and generate out-of-sample predicted probabilities.

- Use cleanlab to identify a list of audio clips with potential label errors.

**Data:** https://www.tensorflow.org/datasets/catalog/spoken_digit

**Pre-Trained Model:** https://huggingface.co/speechbrain/spkrec-xvect-voxceleb

## 1. Install dependencies and import them

To get started, we first need to install the following packages `pip install`:
1. cleanlab
2. speechbrain
3. tensorflow_io

In [ ]:
dependencies = ["cleanlab", "speechbrain", "tensorflow_io"]

if "google.colab" in str(get_ipython()):  # Check if it's running in Google Colab
    %pip install git+https://github.com/weijinglok/cleanlab.git@bdf3a35fbab323105e9bd689ae3aa90064f55d60
    cmd = ' '.join([dep for dep in dependencies if dep != "cleanlab"])
    %pip install $cmd
else:
    missing_dependencies = []
    for dependency in dependencies:
        try:
            __import__(dependency)
        except ImportError:
            missing_dependencies.append(dependency)

    if len(missing_dependencies) > 0:
        print("Missing required dependencies:")
        print(*missing_dependencies, sep=", ")
        print("\nPlease install them before running the rest of this notebook.")

Let's import some of the packages needed throughout this tutorial.

In [ ]:
import os
import pandas as pd
import numpy as np

SEED = 12345
np.random.seed(SEED)  # make experiments reproducible
pd.options.display.max_colwidth = 500

## 2. Load the data

We must first fetch the dataset. To run the below command, you'll need to have `wget` installed; alternatively you can manually navigate to the link in your browser and download from there.

In [ ]:
!wget https://github.com/Jakobovski/free-spoken-digit-dataset/archive/v1.0.9.tar.gz
!mkdir spoken_digits
!tar -xf v1.0.9.tar.gz -C spoken_digits

The audio data are .wav files in the **recordings/** folder. Note that the label for each audio clip (i.e. digit from 0 to 9) is indicated in the prefix of the file name (e.g. **6_nicolas_32.wav** has the label 6).

In [ ]:
DATA_PATH = "spoken_digits/free-spoken-digit-dataset-1.0.9/recordings/"

# Get list of .wav file names
file_paths = []
for (dirpath, dirnames, filenames) in os.walk(DATA_PATH):
    file_paths += [os.path.join(dirpath, file) for file in filenames if file.endswith(".wav")]

# Check out first 3 files
file_paths[:3]

Let's listen to some example audio clips from the dataset. The utility functions below help process the .wav file so we can listen to it in this notebook.

In [ ]:
import tensorflow as tf
import tensorflow_io as tfio
from pathlib import Path
from IPython import display

tf.get_logger().setLevel('ERROR')  # suppress TF warnings 

# Utility function for loading audio files and making sure the sample rate is correct.
@tf.function
def load_wav_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
          file_contents,
          desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

def display_example(wav_file_name, audio_rate=16000):
  """ Allows us to listen to any wav file and displays its given label in the dataset. """
  wav_file_example = load_wav_16k_mono(wav_file_name)
  label = Path(wav_file_name).parts[-1].split("_")[0]
  print(f"Given label for this example: {label}")
  display.display(display.Audio(wav_file_example, rate=audio_rate))


Click the play button below to listen to this example .wav file. Feel free to change the `wav_file_name_example` variable below to listen to other audio clips in the dataset.

In [ ]:
wav_file_name_example = "spoken_digits/free-spoken-digit-dataset-1.0.9/recordings/7_jackson_43.wav"  # change this to hear other examples
display_example(wav_file_name_example)

## 3. Use pre-trained SpeechBrain model to featurize audio

The [SpeechBrain](https://github.com/speechbrain/speechbrain) package  offers many Pytorch neural networks that have been pretrained for speech recognition tasks. Here we instantiate an audio feature extractor using SpeechBrain's `EncoderClassifier()`. We'll use the "spkrec-xvect-voxceleb" network which has been pre-trained on the [VoxCeleb](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/) speech dataset.

In [ ]:
from speechbrain.pretrained import EncoderClassifier

feature_extractor = EncoderClassifier.from_hparams(
  "speechbrain/spkrec-xvect-voxceleb",
  # run_opts={"device":"cuda"}  # Uncomment this to run on GPU if you have one (optional)
)

Next, we run the audio clips through the pre-trained model to extract vector features (aka embeddings).

In [ ]:
# Create dataframe with .wav file names
df = pd.DataFrame(file_paths, columns=["wav_audio_file_path"])
df["label"] = df.wav_audio_file_path.map(lambda x: int(Path(x).parts[-1].split("_")[0]))

df.head(3)

In [ ]:
import torchaudio

def extract_audio_embeddings(model, wav_audio_file_path: str) -> tuple:
    """ Feature extractor that embeds audio into a vector. """
    signal, fs = torchaudio.load(wav_audio_file_path)  # Reformat audio signal into a tensor
    embeddings = model.encode_batch(signal)  # Pass tensor through pretrained neural net and extract representation
    return embeddings

In [ ]:
# Extract audio embeddings
embeddings_list = []
for i, file_name in enumerate(df.wav_audio_file_path): # for each .wav file name
    embeddings = extract_audio_embeddings(feature_extractor, file_name)
    embeddings_list.append(embeddings.cpu().numpy())

embeddings_array = np.squeeze(np.array(embeddings_list))

Now we have our features in a 2D numpy array. Each row in the array corresponds to an audio clip. We're now able to represent each audio clip as a 512-dimensional feature vector!

In [ ]:
print(embeddings_array)
print("Shape of array: ", embeddings_array.shape)

## 4. Fit linear model and compute out-of-sample predicted probabilities

A typical way to leverage pretrained networks for a particular classification task is to add a linear output layer and fine-tune the network parameters on the new data. However this can be computationally intensive. Alternatively, we can freeze the pretrained weights of the network and only train the output layer without having to rely on GPU(s). Here we do this conveniently by fitting a `sklearn` linear model on top of the extracted network embeddings.

To identify label issues, cleanlab requires a probabilistic prediction from your model for every datapoint that should be considered. However these predictions will be *overfit* (and thus unreliable) for datapoints the model was previously trained on. Cleanlab is intended to only be used with **out-of-sample** predicted probabilities, i.e. on datapoints held-out from the model during the training. 

K-fold cross-validation is a straightforward way to produce out-of-sample predicted probabilites for every datapoint in the dataset, by training K copies of our model on different data subsets and using each copy to predict on the subset of data it did not see during training. An additional benefit of cross-validation is that it provides more reliable evaluation of our model than a single training/validation split. We can obtain cross-validated out-of-sample predicted probabilities from any classifier via the [`cross_val_predict`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) wrapper provided in `sklearn`.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

model = LogisticRegression(C=0.01, max_iter=1000, tol=1e-1, random_state=SEED)

# Generate cross-validated predicted probabilities for each datapoint
cv_pred_probs = cross_val_predict(
    estimator=model,
    X=embeddings_array,
    y=df.label.values,
    cv=5,
    method="predict_proba"
)

For each audio clip, the corresponding predicted probabilities in `cv_pred_probs` are produced by a copy of our LogisticRegression model that has never been trained on this audio clip. Hence we call these predictions *out-of-sample*. An additional benefit of cross-validation is that it provides more reliable evaluation of our model than a single training/validation split.

In [ ]:
from sklearn.metrics import accuracy_score

predicted_labels = cv_pred_probs.argmax(axis=1)
cv_accuracy = accuracy_score(df.label.values, predicted_labels)
print(f"Cross-validated estimate of accuracy on held-out data: {cv_accuracy}")

## 5. Use cleanlab to find label issues

Based on the given labels and out-of-sample predicted probabilities,  `cleanlab` can quickly help us identify label issues. Here we request that the indices of the identified label issues should be sorted by cleanlab's *self-confidence* score, which measures the quality of each given label via the probability assigned it in our model's prediction.



In [ ]:
import cleanlab

label_issues_indices = cleanlab.filter.find_label_issues(
    labels=df.label.values,
    pred_probs=cv_pred_probs,
    return_indices_ranked_by="self_confidence",  # ranks the label issues
    )

print(label_issues_indices)

The datapoints flagged by `cleanlab` are those we worth inspecting more closely.

In [ ]:
df.iloc[label_issues_indices]

Let's listen to some audio clips below of label issues that were identified in this list. 

In this example, the given label is the digit **6** but it sounds more like **8**.

In [ ]:
wav_file_name_example = "spoken_digits/free-spoken-digit-dataset-1.0.9/recordings/6_yweweler_14.wav"
display_example(wav_file_name_example)

The following two examples were given the label **6**, but they sound quite ambiguous.

In [ ]:
wav_file_name_example = "spoken_digits/free-spoken-digit-dataset-1.0.9/recordings/6_nicolas_8.wav"
display_example(wav_file_name_example)

In [ ]:
wav_file_name_example = "spoken_digits/free-spoken-digit-dataset-1.0.9/recordings/6_yweweler_35.wav"
display_example(wav_file_name_example)

You can see that even widely-used datasets like Spoken Digit  contain problematic labels. Never blindly trust your data! You should always check it for potential issues, many of which can be easily identified by `cleanlab`.